In [3]:
import os
import re
import emoji
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import unicodedata

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gblasd/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
for n,i in enumerate(['a','b']):
    print(n,i)

0 a
1 b


In [8]:
base_dir = "/Users/gblasd/Documents/DataScience/02_SupervisedLearning/data/spam"
df = None
for i, file in enumerate(os.listdir(base_dir)):
    aux = pd.read_csv(os.path.join(base_dir, file))
    aux["artista"] = file.split(".")[0].split("-")[1]
    if i == 0:
        df = aux.copy()
    else:
        df = pd.concat([df,aux])
df.reset_index(inplace = True, drop = True)

In [ ]:
df.tail(10)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,artista
1946,z12exzcrvpeew1yxg04cd5tbwnmfubnh4kk0k,Anthony1SV,2014-11-14T00:01:37,Please do buy these new Christmas shirts! You ...,1,Psy
1947,z13qyxk5tzq1e5asx22xjt3wdq3ns32f5,Ameenk Chanel,2014-11-14T11:50:02,Free my apps get 1m crdits ! Just click on the...,1,Psy
1948,z12uyzhazxqkzbrzz04cedqpkwzdvjyy3u40k,Brandon Wilson,2014-11-14T12:21:52,Why does a song like this have more views than...,0,Psy
1949,z13sh3cz1kbqgrai504cf53qsq25ypmi5zs0k,Leonel Hernandez,2014-11-14T12:35:38,"Something to dance to, even if your sad JUST ...",0,Psy
1950,z12etrpq5xu0vnm2g230szo5ote3zviny,InterGaming,2014-11-14T13:16:24,everyones back lool this is almost 3 years old...,0,Psy
1951,z13th1q4yzihf1bll23qxzpjeujterydj,Carmen Racasanu,2014-11-14T13:27:52,How can this have 2 billion views when there's...,0,Psy
1952,z13fcn1wfpb5e51xe04chdxakpzgchyaxzo0k,diego mogrovejo,2014-11-14T13:28:08,I don't now why I'm watching this in 2014﻿,0,Psy
1953,z130zd5b3titudkoe04ccbeohojxuzppvbg,BlueYetiPlayz -Call Of Duty and More,2015-05-23T13:04:32,subscribe to me for call of duty vids and give...,1,Psy
1954,z12he50arvrkivl5u04cctawgxzkjfsjcc4,Photo Editor,2015-06-05T14:14:48,hi guys please my android photo editor downloa...,1,Psy
1955,z13vhvu54u3ewpp5h04ccb4zuoardrmjlyk0k,Ray Benich,2015-06-05T18:05:16,The first billion viewed this because they tho...,0,Psy


In [10]:
df.shape

(1956, 6)

In [11]:
df["artista"].value_counts(True)

artista
Eminem       0.229039
LMFAO        0.223926
Shakira      0.189162
KatyPerry    0.178937
Psy          0.178937
Name: proportion, dtype: float64

In [12]:
df["AUTHOR"].value_counts(True)

AUTHOR
M.E.S              0.004090
Louis Bryant       0.003579
5000palo           0.003579
Shadrach Grentz    0.003579
DanteBTV           0.003067
                     ...   
lisabelb           0.000511
said abdesalam     0.000511
Æle Skittles       0.000511
SnEakiESTG         0.000511
Ray Benich         0.000511
Name: proportion, Length: 1792, dtype: float64

In [13]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,artista
0,z13uwn2heqndtr5g304ccv5j5kqqzxjadmc0k,Corey Wilson,2015-05-28T21:39:52.376000,"<a href=""http://www.youtube.com/watch?v=KQ6zr6...",0,LMFAO
1,z124jvczaz3dxhnbc04cffk43oiugj25yzo0k,Epic Gaming,2015-05-28T20:07:20.610000,wierd but funny﻿,0,LMFAO
2,z13tczjy5xj0vjmu5231unho1ofey5zdk,LaS Music,2015-05-28T19:23:35.355000,"Hey guys, I&#39;m a human.<br /><br /><br />Bu...",1,LMFAO
3,z13tzr0hdpnayhqqc04cd3zqqqjkf3ngckk0k,Cheryl Fox,2015-05-28T17:49:35.294000,Party Rock....lol...who wants to shuffle!!!﻿,0,LMFAO
4,z12pcvix4zedcjvyb04ccr1r0mr2g5xwyng0k,PATRICK_TW,2015-05-28T16:28:26.818000,Party rock﻿,0,LMFAO


In [15]:
df.loc[0, "CONTENT"]

'<a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part\ufeff'

### Ingeniería de variables

-> Vectorización ejemplo

                    hola como estas amigo  mal.
hola como estas .    1    1    1     0   1  0 
hola mal amigo       1    0    0     1   0  1
hola mal .           1    0    0     0   1  1

In [16]:
df.head(5)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,artista
0,z13uwn2heqndtr5g304ccv5j5kqqzxjadmc0k,Corey Wilson,2015-05-28T21:39:52.376000,"<a href=""http://www.youtube.com/watch?v=KQ6zr6...",0,LMFAO
1,z124jvczaz3dxhnbc04cffk43oiugj25yzo0k,Epic Gaming,2015-05-28T20:07:20.610000,wierd but funny﻿,0,LMFAO
2,z13tczjy5xj0vjmu5231unho1ofey5zdk,LaS Music,2015-05-28T19:23:35.355000,"Hey guys, I&#39;m a human.<br /><br /><br />Bu...",1,LMFAO
3,z13tzr0hdpnayhqqc04cd3zqqqjkf3ngckk0k,Cheryl Fox,2015-05-28T17:49:35.294000,Party Rock....lol...who wants to shuffle!!!﻿,0,LMFAO
4,z12pcvix4zedcjvyb04ccr1r0mr2g5xwyng0k,PATRICK_TW,2015-05-28T16:28:26.818000,Party rock﻿,0,LMFAO


In [17]:
##Agregar variables
df['len_text_raw'] = df['CONTENT'].str.len()
#url o teléfono 
df['n_urls'] = df.apply(lambda x: re.findall("http://", x["CONTENT"]) + re.findall("https://", x["CONTENT"]), axis=1).str.len()
df['n_phone'] = df['CONTENT'].map( lambda x: re.findall(r'\+?(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})', x)).str.len()
#extraer números pendiente
#malas palabras
#miguel cantidad de palabras
df['n_words'] = df['CONTENT'].str.split(' ').str.len()
df['n_emojis'] = df['CONTENT'].map(emoji.emoji_count)
df['n_email'] = df['CONTENT'].map( lambda x: re.findall(r'[a-z0-9\._-]+@[a-z0-9\._-]+', x.lower())).str.len()

df["n_dots"] = df["CONTENT"].str.split(".").str.len() - 1
df["n_numeral"] = df["CONTENT"].str.split("#").str.len() - 1
df["n_artist"] = df.apply(lambda x:re.findall(x["artista"].lower(), x["CONTENT"].lower()), axis = 1).str.len()
df["n_lower_ratio_len"] = df["CONTENT"].map(lambda x:sum(map(str.islower, x))) / df["len_text_raw"]
df["n_upper_ratio_len"] = df["CONTENT"].map(lambda x:sum(map(str.isupper, x))) / df["len_text_raw"]
df["n_letters"] = df["CONTENT"].map(lambda x:sum(map(str.isalpha, x)))
df["n_lower_ratio_letters"] = df["CONTENT"].map(lambda x:sum(map(str.islower, x))) / df["n_letters"]
df["n_upper_ratio_letters"] = df["CONTENT"].map(lambda x:sum(map(str.isupper, x))) / df["n_letters"]

In [18]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,artista,len_text_raw,n_urls,n_phone,n_words,n_emojis,n_email,n_dots,n_numeral,n_artist,n_lower_ratio_len,n_upper_ratio_len,n_letters,n_lower_ratio_letters,n_upper_ratio_letters
0,z13uwn2heqndtr5g304ccv5j5kqqzxjadmc0k,Corey Wilson,2015-05-28T21:39:52.376000,"<a href=""http://www.youtube.com/watch?v=KQ6zr6...",0,LMFAO,84,1,0,4,0,0,2,0,0,0.559524,0.047619,51,0.921569,0.078431
1,z124jvczaz3dxhnbc04cffk43oiugj25yzo0k,Epic Gaming,2015-05-28T20:07:20.610000,wierd but funny﻿,0,LMFAO,16,0,0,3,0,0,0,0,0,0.812500,0.000000,13,1.000000,0.000000
2,z13tczjy5xj0vjmu5231unho1ofey5zdk,LaS Music,2015-05-28T19:23:35.355000,"Hey guys, I&#39;m a human.<br /><br /><br />Bu...",1,LMFAO,500,1,0,78,0,0,4,3,0,0.672000,0.018000,345,0.973913,0.026087
3,z13tzr0hdpnayhqqc04cd3zqqqjkf3ngckk0k,Cheryl Fox,2015-05-28T17:49:35.294000,Party Rock....lol...who wants to shuffle!!!﻿,0,LMFAO,44,0,0,5,0,0,7,0,0,0.613636,0.045455,29,0.931034,0.068966
4,z12pcvix4zedcjvyb04ccr1r0mr2g5xwyng0k,PATRICK_TW,2015-05-28T16:28:26.818000,Party rock﻿,0,LMFAO,11,0,0,2,0,0,0,0,0,0.727273,0.090909,9,0.888889,0.111111


In [22]:
df["CONTENT"] = df["CONTENT"].map(lambda x:BeautifulSoup(x).text)

In [23]:
def clean_text(text,pattern='[^a-zA-Z0-9]'):
    cleaned_text = unicodedata.normalize('NFD', str(text)).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, ' ', cleaned_text.decode('ascii'), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().lstrip().strip().split())
    return cleaned_text 

In [ ]:
#CountVectorizer?

Init signature:
CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,
)
Docstring:     
Convert a collection of text documents to a matrix of token counts.

This implementation produces a sparse representation of the counts using
scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer
that does some kind of feature selection then the number of features will
be equal to the vocabulary size found by analyzing the data.

For an efficiency comparison of the different feature extractors, see
:ref:`sphx_glr_auto_examples_text_plot_hashing_vs_dict_vectorizer.py`.

Read more in the :ref:`User Guide

ngram_range = (1,5)
Hola como has estado 
____ ____ ___ ______
_________
     _______
          __________
____________________

NO ESTOY MUY BIEN
-> Es importante la relación de palabras porque el contexto puede cambiar, por ejemplo, si nos fijamo sólo en el 'estoy muy bien' y no en el 'no'

In [26]:
stop = set(stopwords.words('english'))
sw=[i for i in stop]

In [27]:
sw

["isn't",
 'if',
 'be',
 'once',
 'was',
 'each',
 'because',
 'at',
 "didn't",
 "haven't",
 "aren't",
 'in',
 'him',
 'such',
 'until',
 "shan't",
 'will',
 'above',
 'after',
 'against',
 'not',
 'more',
 's',
 "they're",
 'this',
 'between',
 'over',
 'with',
 't',
 "you'd",
 'off',
 'too',
 'her',
 'when',
 'here',
 "you've",
 'of',
 'about',
 'so',
 'didn',
 'hers',
 "should've",
 'up',
 'your',
 'doing',
 'has',
 'hasn',
 'but',
 'can',
 'mustn',
 'haven',
 'he',
 'an',
 'as',
 "we're",
 'aren',
 'on',
 'she',
 'by',
 'no',
 'yours',
 'should',
 'nor',
 'while',
 'whom',
 'me',
 'into',
 "i'm",
 'm',
 'do',
 'been',
 'having',
 'have',
 'those',
 'weren',
 'all',
 'than',
 'a',
 "won't",
 'wouldn',
 'ours',
 "weren't",
 'only',
 "we'll",
 'myself',
 'needn',
 "you'll",
 'mightn',
 'herself',
 "he's",
 'now',
 'did',
 'yourself',
 'couldn',
 "it'd",
 'are',
 "shouldn't",
 "that'll",
 'again',
 'who',
 'doesn',
 'i',
 "i've",
 'their',
 'shouldn',
 'theirs',
 'them',
 "i'd",
 'were

In [28]:
cv = TfidfVectorizer(stop_words=sw, ngram_range=(1, 3), min_df=10, max_features=100)

In [29]:
df["txt_CONTENT"] = df["CONTENT"].map(lambda x: clean_text(x))

In [30]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,artista,len_text_raw,n_urls,n_phone,n_words,...,n_email,n_dots,n_numeral,n_artist,n_lower_ratio_len,n_upper_ratio_len,n_letters,n_lower_ratio_letters,n_upper_ratio_letters,txt_CONTENT
0,z13uwn2heqndtr5g304ccv5j5kqqzxjadmc0k,Corey Wilson,2015-05-28T21:39:52.376000,2:19 best part﻿,0,LMFAO,84,1,0,4,...,0,2,0,0,0.559524,0.047619,51,0.921569,0.078431,2 19 best part
1,z124jvczaz3dxhnbc04cffk43oiugj25yzo0k,Epic Gaming,2015-05-28T20:07:20.610000,wierd but funny﻿,0,LMFAO,16,0,0,3,...,0,0,0,0,0.812500,0.000000,13,1.000000,0.000000,wierd but funny
2,z13tczjy5xj0vjmu5231unho1ofey5zdk,LaS Music,2015-05-28T19:23:35.355000,"Hey guys, I'm a human.But I don't want to be a...",1,LMFAO,500,1,0,78,...,0,4,3,0,0.672000,0.018000,345,0.973913,0.026087,hey guys i m a human but i don t want to be a ...
3,z13tzr0hdpnayhqqc04cd3zqqqjkf3ngckk0k,Cheryl Fox,2015-05-28T17:49:35.294000,Party Rock....lol...who wants to shuffle!!!﻿,0,LMFAO,44,0,0,5,...,0,7,0,0,0.613636,0.045455,29,0.931034,0.068966,party rock lol who wants to shuffle
4,z12pcvix4zedcjvyb04ccr1r0mr2g5xwyng0k,PATRICK_TW,2015-05-28T16:28:26.818000,Party rock﻿,0,LMFAO,11,0,0,2,...,0,0,0,0,0.727273,0.090909,9,0.888889,0.111111,party rock


In [31]:
cv.fit(df["txt_CONTENT"])

TfidfVectorizer(max_features=100, min_df=10, ngram_range=(1, 3),
                stop_words=["isn't", 'if', 'be', 'once', 'was', 'each',
                            'because', 'at', "didn't", "haven't", "aren't",
                            'in', 'him', 'such', 'until', "shan't", 'will',
                            'above', 'after', 'against', 'not', 'more', 's',
                            "they're", 'this', 'between', 'over', 'with', 't',
                            "you'd", ...])

In [32]:
ls_features = ["len_text_raw", "n_dots", "n_artist", "n_urls", "n_lower_ratio_len", "n_upper_ratio_len", "n_words", "n_letters", "n_lower_ratio_letters", "n_upper_ratio_letters", "n_emojis", "n_phone", "n_email", ]

In [33]:
X = df[ls_features].fillna(0).join(pd.DataFrame(data = cv.transform(df["txt_CONTENT"]).todense(), columns = cv.get_feature_names_out()))
y = df["CLASS"]

In [34]:
#El vectorizador regresa una matriz sparce, que es una matriz que sólo guarda los campos que tienen valores

In [35]:
X.head()

,len_text_raw,n_dots,n_artist,n_urls,n_lower_ratio_len,n_upper_ratio_len,n_words,n_letters,n_lower_ratio_letters,n_upper_ratio_letters,...,watch,watching,way,website,working,world,would,www,year,youtube
0,84,2,0,1,0.559524,0.047619,4,51,0.921569,0.078431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16,0,0,0,0.812500,0.000000,3,13,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,500,4,0,1,0.672000,0.018000,78,345,0.973913,0.026087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,44,7,0,0,0.613636,0.045455,5,29,0.931034,0.068966,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11,0,0,0,0.727273,0.090909,2,9,0.888889,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
kb = SelectKBest(k=10, score_func=f_classif)

In [37]:
kb.fit(X, y)

SelectKBest()

In [38]:
ls_best = [x for x, y in zip(X.columns, kb.get_support()) if y]

In [39]:
X = X[ls_best]

In [40]:
ls_best

['len_text_raw',
 'n_words',
 'n_letters',
 'channel',
 'check',
 'com',
 'please',
 'song',
 'subscribe',
 'youtube']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [42]:
logreg = LogisticRegression()

In [43]:
logreg.fit(X_train, y_train)

LogisticRegression()

In [44]:
logreg.score(X_test, y_test)

0.901840490797546

In [45]:
ls_scores = cross_val_score(X=X_train, y=y_train, cv = 4, estimator=logreg, scoring="roc_auc")

In [46]:
np.mean(ls_scores), np.std(ls_scores)

(np.float64(0.9501483066453618), np.float64(0.004620612725272415))

In [47]:
#Para aplica el modelo, se utiliza el predict modelo.predict
roc_auc_score(y_score=logreg.predict(X_test), y_true=y_test)

np.float64(0.9012384937238495)

In [48]:
X_test.head()

,len_text_raw,n_words,n_letters,channel,check,com,please,song,subscribe,youtube
1026,49,8,37,0.0,0.000000,0.000000,0.000000,0.332321,0.000000,0.000000
1937,108,22,84,0.0,0.166217,0.000000,0.000000,0.000000,0.223308,0.212245
1734,81,13,61,0.0,0.000000,0.386661,0.000000,0.350220,0.000000,0.000000
1174,492,90,372,0.0,0.000000,0.000000,0.327524,0.000000,0.000000,0.000000
957,25,2,6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
#X_test['y_predicha'] = logreg.predict(X_test)
logreg.predict(X_test)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,

In [50]:
X.describe()

,len_text_raw,n_words,n_letters,channel,check,com,please,song,subscribe,youtube
count,1956.000000,1956.000000,1956.000000,1956.000000,1956.000000,1956.000000,1956.000000,1956.000000,1956.000000,1956.000000
mean,94.700920,16.260736,68.736196,0.037642,0.074865,0.040272,0.036937,0.069069,0.050857,0.044517
std,128.222062,22.669075,93.165963,0.132494,0.156023,0.127648,0.123600,0.195488,0.171132,0.127280
min,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.000000,5.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,48.000000,8.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,97.000000,16.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1200.000000,226.000000,862.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [51]:
logreg.intercept_

array([-1.87882866])

In [55]:
-1.92604009+(b0+b1+b2+b3+b4+b5+b6+b7+b8+b9)

2.497559718464

In [54]:
1/(1+12.15)

0.07604562737642585

In [53]:
b3 = 3.680741*0
b4 = 6.392916*0.644804
b5 = 4.914274*0
b0 = 0.016524*30
b2 = 0.016825*24
b1 = -0.099683*6
b6 = 3.885166*0
b7 = -1.895654*0
b8 = 5.028822*0
b9 = 1.882003*0

In [57]:
X_test

,len_text_raw,n_words,n_letters,channel,check,com,please,song,subscribe,youtube
1026,49,8,37,0.0,0.000000,0.000000,0.000000,0.332321,0.000000,0.000000
1937,108,22,84,0.0,0.166217,0.000000,0.000000,0.000000,0.223308,0.212245
1734,81,13,61,0.0,0.000000,0.386661,0.000000,0.350220,0.000000,0.000000
1174,492,90,372,0.0,0.000000,0.000000,0.327524,0.000000,0.000000,0.000000
957,25,2,6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1024,75,12,61,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20,14,1,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1123,10,3,8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1314,44,8,35,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [58]:
X_test.loc[1026]

len_text_raw    49.000000
n_words          8.000000
n_letters       37.000000
channel          0.000000
check            0.000000
com              0.000000
please           0.000000
song             0.332321
subscribe        0.000000
youtube          0.000000
Name: 1026, dtype: float64

In [59]:
pd.DataFrame(list(zip(ls_best, logreg.coef_[0]))).sort_values(by = 0)

,0,1
3,channel,3.661770
4,check,7.002312
5,com,4.478098
0,len_text_raw,0.022561
2,n_letters,0.013348
1,n_words,-0.113128
6,please,3.447938
7,song,-2.103652
8,subscribe,4.966926
9,youtube,3.039527


In [62]:
pd.to_pickle(logreg, "/Users/gblasd/Documents/DataScience/02_SupervisedLearning/models/logreg2.diplo")

In [65]:
modelo = pd.read_pickle("/Users/gblasd/Documents/DataScience/02_SupervisedLearning/models/logreg2.diplo")

In [66]:
modelo.coef_

array([[ 0.0225607 , -0.11312794,  0.01334777,  3.66176954,  7.00231156,
         4.47809782,  3.44793793, -2.10365185,  4.96692605,  3.03952748]])